# CSV Files and Data Frames

This exercise should get you started with handling data in Python. There is no data analysis yet, just some loading and filtering of the data. 


## Loading data from CSV
Your first step is always accessing the data. Often, the data is stored in a database or within files. One common and generic exchange format for files are Comma Separated Value (CSV) files. The first line of such a file indicates the names of the features, the following lines each contain a single instance.

First, download the ```analcatdata_bankruptcy``` dataset in the data folder and and upload it to your Jupyter notebook. 

Use the cell below to load the data from the CSV file. The data should be loaded into a _data frame_. Data frames are available in python using the ```pandas``` library. In comparison to matrices or similar types, they allow different types of columns, are usually easier to manipulate, e.g., by adding or removing rows/columns, and rows and columns can be named. 

Once you have done this, print some information about the data:
- number of instances
- number of features
- names of the features

## Remove features

If you load all data from a file, you often also load irrelevant features for a task. In case of the data you just loaded, the feature is called ```Company```. This is an ID feature for the instances in the data. Such data must often be removed before further analysis of the data. Your second task is to remove this feature from the data.

In [19]:
import pandas as pd

# Load the bankruptcy dataset
df = pd.read_csv("data/analcatdata_bankruptcy.csv")

# Peek at the data
print("Number of instances (rows):", df.shape[0])
print("Number of features  (cols):", df.shape[1])
print("Feature names:", list(df.columns))
display(df.head())


Number of instances (rows): 50
Number of features  (cols): 7
Feature names: ['Company', 'WC/TA', 'RE/TA', 'EBIT/TA', 'S/TA', 'BVE/BVL', 'Bankrupt']


,Company,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt
0,360Networks,9.3,-7.7,1.6,9.1,3.726,1
1,Advanced_Radio_Telecom,42.6,-60.1,-10.1,0.3,4.130,1
2,Ardent_Communications,-28.8,-203.2,-51.0,14.7,0.111,1
3,At_Home_Corp.,2.5,-433.1,-6.0,29.3,1.949,1
4,Convergent_Communications,26.1,-57.4,-23.5,54.2,0.855,1


In [20]:
# Some copies have 'Company' (ID) — remove it if present
cols_before = set(df.columns)
for candidate in ["Company", "company", "COMPANY"]:
    if candidate in df.columns:
        df = df.drop(columns=[candidate])
        break

print("Removed columns:", list(cols_before - set(df.columns)))
print("Columns now:", list(df.columns))
display(df.head())


Removed columns: ['Company']
Columns now: ['WC/TA', 'RE/TA', 'EBIT/TA', 'S/TA', 'BVE/BVL', 'Bankrupt']


,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt
0,9.3,-7.7,1.6,9.1,3.726,1
1,42.6,-60.1,-10.1,0.3,4.130,1
2,-28.8,-203.2,-51.0,14.7,0.111,1
3,2.5,-433.1,-6.0,29.3,1.949,1
4,26.1,-57.4,-23.5,54.2,0.855,1


## Remove instances with missing values

Real-life data is often not clean, i.e., the data has many problems which must be addressed first, before it can be used for analysis. One common problem are missing features, i.e., not all features are available for all data. This is also the case for the data you just loaded. All missing values are marked as NA in the CSV file. 

Your third task is to remove all instances from the data, that have any missing values and store the remaining instances in a new data frame. If this works correctly, five instances should be removed. You can check this, e.g., by comparing the sizes of the data frames or printing the instances that were removed.


In [21]:
import numpy as np

# Work on a cleaned copy so we keep df for comparison if needed
df_clean = df.copy()

# Treat empty strings/whitespace as missing as well
df_clean = df_clean.replace(r'^\s*$', np.nan, regex=True)

original_rows = df.shape[0]
df_clean = df_clean.dropna()
clean_rows = df_clean.shape[0]

print("Original number of rows:", original_rows)
print("Number of rows after removing missing values:", clean_rows)
print("Number of rows removed:", original_rows - clean_rows)

# Show rows that were removed (from the ORIGINAL df)
removed_rows = df[df.isna().any(axis=1)]
print("\nRows that were removed due to missing values (if any):")
display(removed_rows)

# We'll continue with df_clean for the next tasks


Original number of rows: 50
Number of rows after removing missing values: 50
Number of rows removed: 0

Rows that were removed due to missing values (if any):


,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt


## Computing with data frames

Sometimes you have to compute new columns from the values of existing columns. Please append two new columns to the data frame: The sum of the columns WC/TA and RE/TA and the product of the columns EBIT/TA and S/TA. 

In [22]:
# Create two new columns on the cleaned data
# 1) Sum of WC/TA and RE/TA
# 2) Product of EBIT/TA and S/TA

required_cols = ["WC/TA", "RE/TA", "EBIT/TA", "S/TA"]
missing = [c for c in required_cols if c not in df_clean.columns]
if missing:
    raise ValueError(f"Expected columns missing from dataset: {missing}")

df_feat = df_clean.copy()
df_feat["WC_RE_sum"] = df_feat["WC/TA"] + df_feat["RE/TA"]
df_feat["EBIT_S_prod"] = df_feat["EBIT/TA"] * df_feat["S/TA"]

print("Added columns: ['WC_RE_sum', 'EBIT_S_prod']")
display(df_feat.head())


Added columns: ['WC_RE_sum', 'EBIT_S_prod']


,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt,WC_RE_sum,EBIT_S_prod
0,9.3,-7.7,1.6,9.1,3.726,1,1.6,14.56
1,42.6,-60.1,-10.1,0.3,4.130,1,-17.5,-3.03
2,-28.8,-203.2,-51.0,14.7,0.111,1,-232.0,-749.70
3,2.5,-433.1,-6.0,29.3,1.949,1,-430.6,-175.80
4,26.1,-57.4,-23.5,54.2,0.855,1,-31.3,-1273.70


## Merging data frames

The next task of this exercise is to merge data frames. For this, load the data from the same CSV file as above again. Then merge the data frame with the result from task 2.4, such that:
- the dropped feature from task 2.2 is part of the merged data frame; and
- the removed instances from task 2.3 are still gone; and
- the indirectly computed features from task 2.4 are part of the merged data frame. 

In [23]:
# Reload original, clean in the same way, then merge with df_feat (inner join on index)
df2 = pd.read_csv("data/analcatdata_bankruptcy.csv")

# Drop Company if present
for candidate in ["Company", "company", "COMPANY"]:
    if candidate in df2.columns:
        df2 = df2.drop(columns=[candidate])
        break

# Treat empty strings as NaN and drop rows with missing values to mirror df_clean
df2 = df2.replace(r'^\s*$', np.nan, regex=True).dropna()

# Merge on the index; suffixes to distinguish any duplicate column names
merged_df = pd.merge(df_feat, df2, left_index=True, right_index=True, suffixes=("_1", "_2"))

print("Shapes -> df_feat:", df_feat.shape, ", df2:", df2.shape, ", merged:", merged_df.shape)
display(merged_df.head())


Shapes -> df_feat: (50, 8) , df2: (50, 6) , merged: (50, 14)


,WC/TA_1,RE/TA_1,EBIT/TA_1,S/TA_1,BVE/BVL_1,Bankrupt_1,WC_RE_sum,EBIT_S_prod,WC/TA_2,RE/TA_2,EBIT/TA_2,S/TA_2,BVE/BVL_2,Bankrupt_2
0,9.3,-7.7,1.6,9.1,3.726,1,1.6,14.56,9.3,-7.7,1.6,9.1,3.726,1
1,42.6,-60.1,-10.1,0.3,4.130,1,-17.5,-3.03,42.6,-60.1,-10.1,0.3,4.130,1
2,-28.8,-203.2,-51.0,14.7,0.111,1,-232.0,-749.70,-28.8,-203.2,-51.0,14.7,0.111,1
3,2.5,-433.1,-6.0,29.3,1.949,1,-430.6,-175.80,2.5,-433.1,-6.0,29.3,1.949,1
4,26.1,-57.4,-23.5,54.2,0.855,1,-31.3,-1273.70,26.1,-57.4,-23.5,54.2,0.855,1


## Selecting subsets

Based on the data frame from task 2.5, create new data frames according to the following criteria.
- A data frame with only the rows 10 to 20 and all columns. 
- A data frame with only the columns 1 to 4 and all rows. 
- A data frame with only the columns WC/TA and EBIT/TA and all rows. 
- A data frame with all rows that have the value RE/TA less than -20 and all columns.
- A data frame with all rows that have the value RE/TA less than -20 and bankrupt equal to 0 and all columns. 
- A data frame with all rows that have the value RE/TA less than -20 and bankrupt equal to 0 and only the columns WC/TA and EBIT/TA. 


In [24]:
# Identify the bankruptcy label column (dataset variants may use 'Bankrupt?' or 'Bankrupt')
bank_col = "Bankrupt?" if "Bankrupt?" in df_feat.columns else ("Bankrupt" if "Bankrupt" in df_feat.columns else None)
if bank_col is None:
    raise ValueError("Could not find bankruptcy label column ('Bankrupt?' or 'Bankrupt').")

# a) Rows 10 to 20 (inclusive of 20)
subset_rows = df_feat.iloc[10:21]

# b) Only the bankruptcy label column
subset_bankrupt_col = df_feat[[bank_col]]

# c) Only companies that went bankrupt
subset_only_bankrupt = df_feat[df_feat[bank_col] == 1]

# d) Only companies that did NOT go bankrupt
subset_not_bankrupt = df_feat[df_feat[bank_col] == 0]

print("Subset: rows 10–20")
display(subset_rows)

print("\nSubset: only bankruptcy column")
display(subset_bankrupt_col.head())

print("\nSubset: only bankrupt (label == 1)")
display(subset_only_bankrupt.head())

print("\nSubset: not bankrupt (label == 0)")
display(subset_not_bankrupt.head())


Subset: rows 10–20


,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt,WC_RE_sum,EBIT_S_prod
10,24.6,-29.0,-2.0,21.3,1.968,1,-4.4,-42.60
11,6.6,-50.9,-2.6,28.9,0.258,1,-44.3,-75.14
12,33.9,-46.5,-17.5,0.9,0.828,1,-12.6,-15.75
13,19.1,-66.3,-25.5,22.3,0.460,1,-47.2,-568.65
14,-21.1,-46.0,-26.8,81.4,0.698,1,-67.1,-2181.52
15,2.5,-228.7,-6.7,38.6,0.030,1,-226.2,-258.62
16,47.0,-78.2,-42.0,4.4,0.168,1,-31.2,-184.80
17,9.1,-40.2,-0.7,81.5,0.522,1,-31.1,-57.05
18,43.0,-49.2,-87.4,119.9,2.919,1,-6.2,-10479.26
19,-34.9,-79.0,-13.5,127.8,0.197,1,-113.9,-1725.30



Subset: only bankruptcy column


,Bankrupt
0,1
1,1
2,1
3,1
4,1



Subset: only bankrupt (label == 1)


,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt,WC_RE_sum,EBIT_S_prod
0,9.3,-7.7,1.6,9.1,3.726,1,1.6,14.56
1,42.6,-60.1,-10.1,0.3,4.130,1,-17.5,-3.03
2,-28.8,-203.2,-51.0,14.7,0.111,1,-232.0,-749.70
3,2.5,-433.1,-6.0,29.3,1.949,1,-430.6,-175.80
4,26.1,-57.4,-23.5,54.2,0.855,1,-31.3,-1273.70



Subset: not bankrupt (label == 0)


,WC/TA,RE/TA,EBIT/TA,S/TA,BVE/BVL,Bankrupt,WC_RE_sum,EBIT_S_prod
25,30.6,-14.4,-4.9,2.2,3.482,0,16.2,-10.78
26,9.8,-33.8,-7.1,3.2,5.965,0,-24.0,-22.72
27,37.8,-45.4,-7.1,17.1,3.450,0,-7.6,-121.41
28,2.2,31.6,22.0,58.0,2.758,0,33.8,1276.00
29,-11.5,27.6,24.4,51.4,2.266,0,16.1,1254.16
